In [2]:
from osgeo import ogr, gdal


tiff_path = r'C:\Users\xdh\Documents\自然资源部\光伏2.tif'
tiff_ds = gdal.Open(tiff_path)
transform = tiff_ds.GetGeoTransform()

# 打开shp文件
path=r"C:\Users\xdh\Documents\自然资源部\风机标注样本成果\風機name_shp\output_file_F49E021010-DOM-202004.shp"
shp = ogr.Open(path)
layer = shp.GetLayer()

# 获取空间参考和边界框信息
srs = layer.GetSpatialRef()
xmin, xmax, ymin, ymax = layer.GetExtent()
print(layer.GetExtent())
print(transform[1])

AttributeError: 'NoneType' object has no attribute 'GetGeoTransform'

In [2]:
import numpy as np

# 将边界框信息转换为像素坐标
pixel_size = transform[1]  # 像素大小
xres = yres = pixel_size
xoff, yoff = xmin, ymax
cols = int(np.ceil((xmax - xmin) / pixel_size))
rows = int(np.ceil((ymax - ymin) / pixel_size))


In [3]:
from shapely.geometry import Polygon
from shapely.ops import transform

# 定义转换函数
def shp_to_pixel_coord(x, y):
    """将shp文件中的坐标转换为像素坐标"""
    x_pix = int((x - xoff) / xres)
    y_pix = int((yoff - y) / yres)
    return x_pix, y_pix

# 将每个几何图形转换为图像中的像素坐标
pixel_coords = []
for feature in layer:
    geom = feature.GetGeometryRef()
    if geom is None:
        continue
    if geom.GetGeometryType() == ogr.wkbPolygon:
        # 转换为Shapely的Polygon对象
        exterior_ring = geom.GetGeometryRef(0)
        # pt=exterior_ring.GetPoints()
        # print(pt)
        # for pt in exterior_ring.GetPoints():
        #     print(pt)
        #     # poly = Polygon(pt)
        poly = Polygon([(pt[0],pt[1]) for pt in exterior_ring.GetPoints()])
        # 转换为像素坐标
        poly = transform(shp_to_pixel_coord, poly)
        pixel_coords.append(poly.exterior.coords[:-1])


In [4]:
import cv2

# 创建新的图像
image = np.zeros((rows, cols), dtype=np.uint8)

# 绘制几何图形
for coords in pixel_coords:
    pts = np.array(coords, np.int32)
    cv2.fillPoly(image, [pts], color=(255, 255, 255))

# 将图像转换为二值图像
thresh, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY)
# cv2.imshow('image', binary_image)

In [5]:
cv2.imwrite('binary_image.png', binary_image)


True

In [7]:
block_size = 512

# 切分
for i in range(0, rows, block_size):
    for j in range(0, cols, block_size):
        # 提取块
        block = binary_image[i:i+block_size, j:j+block_size]
        # 忽略空白块
        if block.sum() == 0:
            continue
        # 调整块大小
        block = cv2.resize(block, (block_size, block_size), interpolation=cv2.INTER_NEAREST)
        # 保存块
        # filename = 'block_{i}_{j}.png'
        # cv2.imwrite(filename, block)
        cv2.imwrite(f'block_{i}_{j}.png', block)

In [ ]:
import cv2
import numpy as np

# Load image
img = cv2.imread('image.png', cv2.IMREAD_GRAYSCALE)

# Get image width and height
height, width = img.shape[:2]

# Define block size
block_size = 512

# Calculate number of rows and columns
rows = int(np.ceil(height / block_size))
cols = int(np.ceil(width / block_size))

# Calculate new image dimensions
new_height = rows * block_size
new_width = cols * block_size

# Create new image and fill with zeros
new_img = np.zeros((new_height, new_width), dtype=np.uint8)

# Copy original image to new image
new_img[:height, :width] = img

# Loop through image and extract blocks
for y in range(0, new_height, block_size):
    for x in range(0, new_width, block_size):
        # Calculate block coordinates
        x1 = x
        y1 = y
        x2 = x + block_size
        y2 = y + block_size

        # Extract block from image
        block = new_img[y1:y2, x1:x2]

        # Process block
        block = cv2.threshold(block, 127, 255, cv2.THRESH_BINARY)[1]

        # Save block to file
        filename = f'block_{x1}_{y1}.png'
        cv2.imwrite(filename, block)
